In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

image_size = 128

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# paths (adjust only if your folders differ)
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "images", "images_bottle")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR  = os.path.join(DATA_DIR, "test")

train_dataset = datasets.ImageFolder(
    root=TRAIN_DIR,
    transform=transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

print("Total training images:", len(train_dataset))
print("Classes:", train_dataset.classes)

import matplotlib.pyplot as plt
import torchvision

def show_images(loader, n=8):
    imgs, _ = next(iter(loader))
    grid = torchvision.utils.make_grid(imgs[:n], nrow=4)
    plt.figure(figsize=(6,6))
    plt.imshow(grid.permute(1,2,0))
    plt.axis("off")
    plt.title("Sample Training Images")
    plt.show()

show_images(train_loader)

import torch.nn as nn

class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1),   # 128 → 64
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),  # 64 → 32
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1), # 32 → 16
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),  # 16 → 32
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),   # 32 → 64
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, 2, 1),    # 64 → 128
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)
    model = ConvAutoEncoder().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    print(model)

    epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for imgs, _ in train_loader:
        imgs = imgs.to(device)

        recon = model(imgs)
        loss = criterion(recon, imgs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}]  Loss: {running_loss/len(train_loader):.6f}")

def compute_anomaly_scores(model, loader):
    model.eval()
    scores = []

    with torch.no_grad():
        for imgs, _ in loader:
            imgs = imgs.to(device)
            recon = model(imgs)

            # per-image MSE
            error = torch.mean((imgs - recon) ** 2, dim=[1,2,3])
            scores.extend(error.cpu().numpy())

    return scores
# training scores (used for threshold)
train_scores = compute_anomaly_scores(model, train_loader)
print("Train anomaly scores (first 5):", train_scores[:5])


import numpy as np

mean_score = np.mean(train_scores)
std_score = np.std(train_scores)

threshold = mean_score + 3 * std_score
print("Anomaly threshold:", threshold)

test_dataset = datasets.ImageFolder(
    root=TEST_DIR,
    transform=transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False
)

test_scores = compute_anomaly_scores(model, test_loader)

preds = ["ANOMALY" if s > threshold else "NORMAL" for s in test_scores]

print("Test scores (first 10):", test_scores[:10])
print("Predictions (first 10):", preds[:10])

def visualize_reconstruction(model, loader, threshold, n=6):
    model.eval()
    imgs, _ = next(iter(loader))
    imgs = imgs[:n].to(device)

    with torch.no_grad():
        recon = model(imgs)
        errors = torch.mean((imgs - recon) ** 2, dim=[1,2,3])

    plt.figure(figsize=(12,4))
    for i in range(n):
        # Original
        plt.subplot(2, n, i+1)
        plt.imshow(imgs[i].cpu().permute(1,2,0))
        plt.title("Orig")
        plt.axis("off")

        # Reconstructed
        plt.subplot(2, n, i+1+n)
        plt.imshow(recon[i].cpu().permute(1,2,0))
        label = "Anomaly" if errors[i] > threshold else "Normal"
        plt.title(label)
        plt.axis("off")

    plt.show()

visualize_reconstruction(model, test_loader, threshold)










